<a href="https://colab.research.google.com/github/Gaabiin5/Hugging_Face_Formation/blob/main/monitoring_and_evaluating_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bonus Unit 1: Observability and Evaluation of Agents

In this tutorial, we will learn how to **monitor the internal steps (traces) of our AI agent** and **evaluate its performance** using open-source observability tools.

The ability to observe and evaluate an agent’s behavior is essential for:
- Debugging issues when tasks fail or produce suboptimal results
- Monitoring costs and performance in real-time
- Improving reliability and safety through continuous feedback

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course/unit1/introduction).

## Exercise Prerequisites 🏗️

Before running this notebook, please be sure you have:

🔲 📚  **Studied** [Introduction to Agents](https://huggingface.co/learn/agents-course/unit1/introduction)

🔲 📚  **Studied** [The smolagents framework](https://huggingface.co/learn/agents-course/unit2/smolagents/introduction)

## Step 0: Install the Required Libraries

We will need a few libraries that allow us to run, monitor, and evaluate our agents:

In [1]:
%pip install langfuse 'smolagents[telemetry]' openinference-instrumentation-smolagents datasets 'smolagents[gradio]' gradio --upgrade

Note: you may need to restart the kernel to use updated packages.


## Step 1: Instrument Your Agent

In this notebook, we will use [Langfuse](https://langfuse.com/) as our observability tool, but you can use **any other OpenTelemetry-compatible service**. The code below shows how to set environment variables for Langfuse (or any OTel endpoint) and how to instrument your smolagent.

**Note:** If you are using LlamaIndex or LangGraph, you can find documentation on instrumenting them [here](https://langfuse.com/docs/integrations/llama-index/workflows) and [here](https://langfuse.com/docs/integrations/langchain/example-python-langgraph).

In [2]:
import os
from dotenv import load_dotenv

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Récupérer les variables
langfuse_public_key = os.getenv("LANGFUSE_PUBLIC_KEY")
langfuse_secret_key = os.getenv("LANGFUSE_SECRET_KEY")
langfuse_host = os.getenv("LANGFUSE_HOST")
hf_token = os.getenv("HF_TOKEN")

# os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region


With the environment variables set, we can now initialize the Langfuse client. get_client() initializes the Langfuse client using the credentials provided in the environment variables.

In [3]:
from langfuse import get_client

langfuse = get_client()

# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


In [4]:
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

SmolagentsInstrumentor().instrument()

## Step 2: Test Your Instrumentation

Here is a simple CodeAgent from smolagents that calculates `1+1`. We run it to confirm that the instrumentation is working correctly. If everything is set up correctly, you will see logs/spans in your observability dashboard.

In [5]:
!pip install smolagents

In [11]:
from smolagents.models import ChatMessage
from groq import Groq

class GroqSmolModel:
    def __init__(self, model_name="qwen-qwq-32b", temperature=0):
        self.client = Groq()
        self.model_name = model_name
        self.temperature = temperature

    def generate(self, input_messages, stop_sequences=None, **kwargs):
        ALLOWED_ROLES = {"system", "user", "assistant"}

        def map_role(role):
            if role in ALLOWED_ROLES:
                return role
            return "system"  # fallback safe

        messages_payload = []
        for m in input_messages:
            role = map_role(m.role)
            content = m.content or ""
            if not isinstance(content, str):
                content = str(content)
            messages_payload.append({"role": role, "content": content})

        print("=== Final messages_payload ===")
        for m in messages_payload:
            print(m)

        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=messages_payload,
            temperature=self.temperature,
            stop=stop_sequences
        )

        content = response.choices[0].message.content

        return ChatMessage(role="assistant", content=content)


In [12]:
from smolagents import InferenceClientModel, CodeAgent

# Create a simple agent to test instrumentation
agent = CodeAgent(
    tools=[],
    # model=InferenceClientModel() # Npt enough credit
    model=GroqSmolModel()
)

agent.run("1+1=")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 1+1=                                                                                                            │
│                                                                                                                 │
╰─ GroqSmolModel -  ──────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

=== Final messages_payload ===
{'role': <MessageRole.SYSTEM: 'system'>, 'content': '[{\'type\': \'text\', \'text\': \'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \\\'Thought:\\\', \\\'<code>\\\', and \\\'Observation:\\\' sequences.\\n\\nAt each step, in the \\\'Thought:\\\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\\nThen in the \\\'<code>\\\' sequence, you should write the code in simple Python. The code sequence must end with \\\'</code>\\\' sequence.\\nDuring each intermediate step, you can use \\\'print()\\\' to save whatever important information you will then need.\\nThese print outputs will then appear in the \\\'Obser

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = 1 + 1                                                                                                   
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2

[Step 1: Duration 1.15 seconds]

2

Check your [Langfuse Traces Dashboard](https://cloud.langfuse.com/traces) (or your chosen observability tool) to confirm that the spans and logs have been recorded.

Example screenshot from Langfuse:

![Example trace in Langfuse](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/first-example-trace.png)

_[Link to the trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/1b94d6888258e0998329cdb72a371155?timestamp=2025-03-10T11%3A59%3A41.743Z)_

## Step 3: Observe and Evaluate a More Complex Agent

Now that you have confirmed your instrumentation works, let's try a more complex query so we can see how advanced metrics (token usage, latency, costs, etc.) are tracked.

In [13]:
from smolagents import (CodeAgent, DuckDuckGoSearchTool, InferenceClientModel)

search_tool = DuckDuckGoSearchTool()
agent = CodeAgent(tools=[search_tool], model=GroqSmolModel())

agent.run("How many Rubik's Cubes could you fit inside the Notre Dame Cathedral?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How many Rubik's Cubes could you fit inside the Notre Dame Cathedral?                                           │
│                                                                                                                 │
╰─ GroqSmolModel -  ──────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

=== Final messages_payload ===
{'role': <MessageRole.SYSTEM: 'system'>, 'content': '[{\'type\': \'text\', \'text\': \'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \\\'Thought:\\\', \\\'<code>\\\', and \\\'Observation:\\\' sequences.\\n\\nAt each step, in the \\\'Thought:\\\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\\nThen in the \\\'<code>\\\' sequence, you should write the code in simple Python. The code sequence must end with \\\'</code>\\\' sequence.\\nDuring each intermediate step, you can use \\\'print()\\\' to save whatever important information you will then need.\\nThese print outputs will then appear in the \\\'Obser

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Find Notre Dame Cathedral's volume or dimensions                                                       
  cathedral_info = web_search("Notre Dame Cathedral Paris internal volume dimensions")                             
  print("Cathedral info search result:", cathedral_info)                                                           
                                                                                                                   
  # Step 2: If volume isn't found, get length, width, height from search results                                   
  # Assume from results: Length ~130m, Width ~48m, Height (nave) ~35m. Calculate volume as length * width *        
  height                                                                                                           
  length = 130  # meters                                                                                           
  width = 48                                                                                                       
  height = 35                                                                                                      
  volume_cathedral = length * width * height                                                                       
  print(f"Cathedral volume estimate: {volume_cathedral} m³")                                                       
                                                                                                                   
  # Step 3: Rubik's Cube volume (5.6cm = 0.056m per edge)                                                          
  cube_side_m = 0.056                                                                                              
  volume_cube = cube_side_m ** 3                                                                                   
  print(f"Cube volume: {volume_cube:.2e} m³")                                                                      
                                                                                                                   
  # Step 4: Compute total cubes = cathedral_volume / cube_volume                                                   
  total_cubes = volume_cathedral / volume_cube                                                                     
  final_answer(int(total_cubes))                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Cathedral info search result: ## Search Results

[Interior Architecture - Notre-Dame de 
Paris](https://www.notredamedeparis.fr/en/understand/architecture/interior-architecture/)
The interior architecture on three levels is characterized by ribbed vaults, a double ambulatory, radiating 
chapels, and stained glass windows. ... This volume is proportional to the dimensions of the cathedral. Liturgical 
celebrations take place in this space. The two arms of the transept are also called transept arms. In the 13th 
century ...

[cathedral plans, and facts | France zone at abelard.org](https://www.abelard.org/france/cathedral_plans_facts.php)
internal volume : 200,000 m³ ... La cathédrale de Notre Dame de Paris . cathedral dimensions • overall length : 130
m / 430 ft • overall width : 32 m / 105 ft • nave, length : 73 m • transept width : 46 m / 151 ft • height, choir 
vaulting : 33.5 m. See also the fire at the cathedral ...

[List of largest church buildings - Wikipedia](https://en.wikipedia.org/wiki/List_of_largest_church_buildings)
The longest Gothic cathedral in Europe [67] Notre Dame de Paris: 4,800 [citation needed] 5,500 9,000 1163-1345; 
2019-present (reconstruction) Paris: France: Catholic Almudena Cathedral: 4,800 [citation needed] 1883-1993 Madrid:
Spain: Catholic It has a north-south orientation instead of east-west. Dresden Cathedral: 4,800 [citation ...

[How Big Is The Notre Dame Cathedral - 
Christian.net](https://christian.net/arts-and-culture/how-big-is-the-notre-dame-cathedral/)
The Notre Dame Cathedral, a timeless symbol of Paris, stands as a testament to the ingenuity and artistry of human 
civilization. ... Interior Volume. The voluminous interior of the Notre Dame Cathedral encompasses a space that 
resonates with the echoes of centuries past, spanning a length of over 400 feet (122 meters) and a width of ...

[Notre Dame Cathedral - Data, Photos & Plans - 
WikiArquitectura](https://en.wikiarquitectura.com/building/Notre-Dame-Cathedral/)
The cathedral underwent many changes and restorations, the most significant undertaken in 1844 by Viollet-le-Duc 
and Baptiste Lassus, which lasted 23 years. More recently, in 1965, excavations under the cathedral revealed the 
existence of the catacombs of Roman and medieval rooms. Victor Hugo wrote in his 1831 novel "Notre Dame de Paris".

[Notre-Dame de Paris | History, Style, Fire, & Facts | 
Britannica](https://www.britannica.com/topic/Notre-Dame-de-Paris)
Notre-Dame Cathedral consists of a choir and apse, a short transept, and a nave flanked by double aisles and square
chapels.Its central spire was added during restoration in the 19th century, replacing the original, which had been 
completely removed in the 18th century because of instability. The interior of the cathedral is 427 by 157 feet 
(130 by 48 meters) in plan, and the roof is 115 feet ...

[The Majestic Layout of Notre-Dame - Friends of Notre-Dame de 
Paris](https://www.friendsofnotredamedeparis.org/notre-dame-cathedral/architecture/layout/)
Like most French cathedrals, Notre-Dame de Paris draws a plan in the shape of a Latin cross. Its main facade is 
oriented west-north-west, and the apse is oriented east-south-east. The cathedral can hold up to 9,000 people, 
including 1,500 in the stands. Notre Dame floor plans Notre Dame floor plans Notre-Dame Cathedral Dimensions

[The Cathedral of Notre Dame in Paris, France, covers an area of 4,800 ...](https://brainly.com/question/51016375)
The volume of Notre Dame Cathedral is approximately 206,400 cubic meters. To find the volume of the interior of the
cathedral, we can use the formula for the volume of a rectangular prism: Volume = Area × Height. Step-by-step 
explanation: Given area: 4,800 square meters; Given height: 43 meters; Volume = 4,800 m² × 43 m; Volume = 206,400 
...

[Notre-Dame : Dimensions - Club des Amis du 
MECCANO](https://www.club-amis-meccano.org/projets-dexception/la-cathedrale-de-paris/notre-dame-dimensions)
Notre-Dame : Dimensi

[Step 1: Duration 7.57 seconds]

1243622448

### Trace Structure

Most observability tools record a **trace** that contains **spans**, which represent each step of your agent’s logic. Here, the trace contains the overall agent run and sub-spans for:
- The tool calls (DuckDuckGoSearchTool)
- The LLM calls (InferenceClientModel)

You can inspect these to see precisely where time is spent, how many tokens are used, and so on:

![Trace tree in Langfuse](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/trace-tree.png)

_[Link to the trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/1ac33b89ffd5e75d4265b62900c348ed?timestamp=2025-03-07T13%3A45%3A09.149Z&display=preview)_

## Online Evaluation

In the previous section, we learned about the difference between online and offline evaluation. Now, we will see how to monitor your agent in production and evaluate it live.

### Common Metrics to Track in Production

1. **Costs** — The smolagents instrumentation captures token usage, which you can transform into approximate costs by assigning a price per token.
2. **Latency** — Observe the time it takes to complete each step, or the entire run.
3. **User Feedback** — Users can provide direct feedback (thumbs up/down) to help refine or correct the agent.
4. **LLM-as-a-Judge** — Use a separate LLM to evaluate your agent’s output in near real-time (e.g., checking for toxicity or correctness).

Below, we show examples of these metrics.

#### 1. Costs

Below is a screenshot showing usage for `Qwen2.5-Coder-32B-Instruct` calls. This is useful to see costly steps and optimize your agent.

![Costs](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/smolagents-costs.png)

_[Link to the trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/1ac33b89ffd5e75d4265b62900c348ed?timestamp=2025-03-07T13%3A45%3A09.149Z&display=preview)_

#### 2. Latency

We can also see how long it took to complete each step. In the example below, the entire conversation took 32 seconds, which you can break down by step. This helps you identify bottlenecks and optimize your agent.

![Latency](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/smolagents-latency.png)

_[Link to the trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/1ac33b89ffd5e75d4265b62900c348ed?timestamp=2025-03-07T13%3A45%3A09.149Z&display=preview)_

#### 3. Additional Attributes

You may also pass additional attributes to your spans. These can include `user_id`, `tags`, `session_id`, and custom metadata. Enriching traces with these details is important for analysis, debugging, and monitoring of your application’s behavior across different users or sessions.

In [14]:
from smolagents import (CodeAgent, DuckDuckGoSearchTool, InferenceClientModel)

search_tool = DuckDuckGoSearchTool()
agent = CodeAgent(
    tools=[search_tool],
    model=GroqSmolModel()
)

with langfuse.start_as_current_span(
    name="Smolagent-Trace",
    ) as span:

    # Run your application here
    response = agent.run("What is the capital of Germany?")

    # Pass additional attributes to the span
    span.update_trace(
        input="What is the capital of Germany?",
        output=response,
        user_id="smolagent-user-123",
        session_id="smolagent-session-123456789",
        tags=["city-question", "testing-agents"],
        metadata={"email": "user@langfuse.com"},
        )

# Flush events in short-lived applications
langfuse.flush()

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the capital of Germany?                                                                                 │
│                                                                                                                 │
╰─ GroqSmolModel -  ──────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

=== Final messages_payload ===
{'role': <MessageRole.SYSTEM: 'system'>, 'content': '[{\'type\': \'text\', \'text\': \'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \\\'Thought:\\\', \\\'<code>\\\', and \\\'Observation:\\\' sequences.\\n\\nAt each step, in the \\\'Thought:\\\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\\nThen in the \\\'<code>\\\' sequence, you should write the code in simple Python. The code sequence must end with \\\'</code>\\\' sequence.\\nDuring each intermediate step, you can use \\\'print()\\\' to save whatever important information you will then need.\\nThese print outputs will then appear in the \\\'Obser

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = web_search(query="What is the capital of Germany?")                                                     
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Capital of Germany - Wikipedia](https://en.wikipedia.org/wiki/Capital_of_Germany)
The capital of Germany is the city of Berlin. [1] It is the seat of the Federal President of Germany, whose 
official residence is Bellevue Castle. The Federal Council is the representation of the Lands of Germany and has 
its seat at the former Prussian House of Lords.

[Berlin | History, Map, Population, Attractions, & Facts | Britannica](https://www.britannica.com/place/Berlin)
Berlin is the capital and chief urban center of Germany. The city lies at the heart of the North German Plain, 
athwart an east-west commercial and geographic axis that helped make it the capital of the kingdom of Prussia and 
then, from 1871, of a unified Germany.

[What Is The Capital Of Germany? - 
WorldAtlas](https://www.worldatlas.com/articles/what-is-the-capital-of-germany.html)
Learn about Berlin, the largest and most populous city in Germany, and its role as the capital of the Federal 
Republic of Germany. Discover its history from its foundation to its division and reunification, and its 
attractions as a cultural and political center.

[What is the Capital of Germany? - Mappr](https://www.mappr.co/capital-cities/germany/)
Berlin is the capital of Germany and a cosmopolitan city with a rich and turbulent history. Learn about its 
location, landmarks, culture, climate, population, and industries on this web page.

[List of Germany States and Capitals, Map - Whereig.com](https://www.whereig.com/germany/states-and-capitals.html)
Find out the names and locations of the 16 states and capitals of Germany, including Berlin, Munich, Hamburg and 
more. See a map of Germany and learn about its history, culture and geography.

[What is the Capital of Germany? - Maps of 
World](https://www.mapsofworld.com/answers/world/what-is-the-capital-of-germany/)
Capital of Germany - Berlin. Berlin is the capital city of Germany. Berlin has lots of history and interesting 
things to see, like the Brandenburg Gate and the Berlin Wall Memorial. It's also where important government 
decisions are made and where new ideas in art and technology often start. ...

[Berlin - Simple English Wikipedia, the free encyclopedia](https://simple.wikipedia.org/wiki/Berlin)
Berlin (/ b ɜːr ˈ l ɪ n /; German: [bɛʁˈliːn] [7]) is the capital city of Germany.It is the largest city in the 
European Union by population, with around 3.9 million people in 2024. [8] Berlin residents come from about 190 
different countries. The city is in the eastern part of Germany in Central Europe and is surrounded by many forests
and lakes.

[Berlin - the city where anything is possible - Germany 
Travel](https://www.germany.travel/en/inspiring-germany/berlin.html)
Berlin is the seat of the German parliament and home to many historical and artistic landmarks. It is also a trendy
and green destination with a vibrant nightlife and a relaxed atmosphere.

[Capital of the Federal Republic of Germany](https://knowgermany.com/capital-of-the-federal-republic-of-germany/)
Learn about the history, landmarks, and culture of Berlin, the capital of Germany. Find out why Berlin was chosen 
as the capital of a reunified Germany and how it became a symbol of unity and a hub for art and innovation.

[Germany's Capital Moves From Bonn to Berlin - 
ThoughtCo](https://www.thoughtco.com/germany-capital-from-bonn-to-berlin-1434930)
A Vote to Decide the Capital . With the raising of the German flag on October 3, 1990, the two former countries 
(the German Democratic Republic and the Federal Republic of Germany) merged to become one unified Germany.

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

=== Final messages_payload ===
{'role': <MessageRole.SYSTEM: 'system'>, 'content': '[{\'type\': \'text\', \'text\': \'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \\\'Thought:\\\', \\\'<code>\\\', and \\\'Observation:\\\' sequences.\\n\\nAt each step, in the \\\'Thought:\\\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\\nThen in the \\\'<code>\\\' sequence, you should write the code in simple Python. The code sequence must end with \\\'</code>\\\' sequence.\\nDuring each intermediate step, you can use \\\'print()\\\' to save whatever important information you will then need.\\nThese print outputs will then appear in the \\\'Obser

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Berlin")                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Berlin

[Step 2: Duration 0.83 seconds]

![Enhancing agent runs with additional metrics](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/smolagents-attributes.png)

#### 4. User Feedback

If your agent is embedded into a user interface, you can record direct user feedback (like a thumbs-up/down in a chat UI). Below is an example using [Gradio](https://gradio.app/) to embed a chat with a simple feedback mechanism.

In the code snippet below, when a user sends a chat message, we capture the trace in Langfuse. If the user likes/dislikes the last answer, we attach a score to the trace.

In [ ]:
import gradio as gr
from smolagents import (CodeAgent, InferenceClientModel)
from langfuse import get_client

langfuse = get_client()

model = GroqSmolModel()
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

trace_id = None

def respond(prompt, history):
    with langfuse.start_as_current_span(
        name="Smolagent-Trace"):

        # Run your application here
        output = agent.run(prompt)

        global trace_id
        trace_id = langfuse.get_current_trace_id()

    history.append({"role": "assistant", "content": str(output)})
    return history

def handle_like(data: gr.LikeData):
    # For demonstration, we map user feedback to a 1 (like) or 0 (dislike)
    if data.liked:
        langfuse.create_score(
            value=1,
            name="user-feedback",
            trace_id=trace_id
        )
    else:
        langfuse.create_score(
            value=0,
            name="user-feedback",
            trace_id=trace_id
        )

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat", type="messages")
    prompt_box = gr.Textbox(placeholder="Type your message...", label="Your message")

    # When the user presses 'Enter' on the prompt, we run 'respond'
    prompt_box.submit(
        fn=respond,
        inputs=[prompt_box, chatbot],
        outputs=chatbot
    )

    # When the user clicks a 'like' button on a message, we run 'handle_like'
    chatbot.like(handle_like, None, None)

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 1+1                                                                                                             │
│                                                                                                                 │
╰─ GroqSmolModel -  ──────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

=== Final messages_payload ===
{'role': <MessageRole.SYSTEM: 'system'>, 'content': '[{\'type\': \'text\', \'text\': \'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \\\'Thought:\\\', \\\'<code>\\\', and \\\'Observation:\\\' sequences.\\n\\nAt each step, in the \\\'Thought:\\\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\\nThen in the \\\'<code>\\\' sequence, you should write the code in simple Python. The code sequence must end with \\\'</code>\\\' sequence.\\nDuring each intermediate step, you can use \\\'print()\\\' to save whatever important information you will then need.\\nThese print outputs will then appear in the \\\'Obser

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = 1 + 1                                                                                                   
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2

[Step 1: Duration 1.28 seconds]

User feedback is then captured in your observability tool:

![User feedback is being captured in Langfuse](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/user-feedback-gradio.png)

#### 5. LLM-as-a-Judge

LLM-as-a-Judge is another way to automatically evaluate your agent's output. You can set up a separate LLM call to gauge the output’s correctness, toxicity, style, or any other criteria you care about.

**Workflow**:
1. You define an **Evaluation Template**, e.g., "Check if the text is toxic."
2. Each time your agent generates output, you pass that output to your "judge" LLM with the template.
3. The judge LLM responds with a rating or label that you log to your observability tool.

Example from Langfuse:

![LLM-as-a-Judge Evaluation Template](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/evaluator-template.png)
![LLM-as-a-Judge Evaluator](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/evaluator.png)

In [16]:
# Example: Checking if the agent’s output is toxic or not.
from smolagents import (CodeAgent, DuckDuckGoSearchTool, InferenceClientModel)

search_tool = DuckDuckGoSearchTool()
agent = CodeAgent(tools=[search_tool], model=GroqSmolModel())

agent.run("Can eating carrots improve your vision?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can eating carrots improve your vision?                                                                         │
│                                                                                                                 │
╰─ GroqSmolModel -  ──────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

=== Final messages_payload ===
{'role': <MessageRole.SYSTEM: 'system'>, 'content': '[{\'type\': \'text\', \'text\': \'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \\\'Thought:\\\', \\\'<code>\\\', and \\\'Observation:\\\' sequences.\\n\\nAt each step, in the \\\'Thought:\\\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\\nThen in the \\\'<code>\\\' sequence, you should write the code in simple Python. The code sequence must end with \\\'</code>\\\' sequence.\\nDuring each intermediate step, you can use \\\'print()\\\' to save whatever important information you will then need.\\nThese print outputs will then appear in the \\\'Obser

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Search for reliable sources on carrots and vision improvement                                                  
  results = web_search("do carrots improve vision scientific evidence")                                            
  print("Search results:", results)                                                                                
                                                                                                                   
  # Check if the results mention vitamin A's role or clarify the myth                                              
  if "vitamin A" in results and "night vision" in results:                                                         
      conclusion = "Carrots provide vitamin A essential for eye health, but do not significantly improve vision    
  beyond preventing deficiencies."                                                                                 
  else:                                                                                                            
      conclusion = "No, according to sources, carrots do not enhance vision beyond normal levels."                 
                                                                                                                   
  final_answer(conclusion)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Search results: ## Search Results

[Fact or Fiction?: Carrots Improve Your Vision - Scientific 
American](https://www.scientificamerican.com/article/fact-or-fiction-carrots-improve-your-vision/)
In settings where undernourished people suffer from extreme vitamin A deficiencies, such as Nepal or India, 
supplements of the vitamin or beta-carotene have been shown to improve night vision. But ...

[Carrots & Eye Health: Myth or Fact? - University of Utah 
Health](https://healthcare.utah.edu/healthfeed/2025/04/carrots-eye-health-myth-or-fact)
You've probably heard that carrots are good for your eyes, and it's true. Carrots are a super source of 
beta-carotene—and that's a start. But it takes a whole lot more to protect your irreplaceable vision. In an ideal 
eye-healthy world, you wear UV-blocking shades and put them on your kids, get ...

[Myth or Fact: Eating Carrots Improves Eyesight - Duke 
Health](https://www.dukehealth.org/blog/myth-or-fact-eating-carrots-improves-eyesight)
Before you run to the store to stock up on bags of carrots, keep in mind that overdosing on carrots in an attempt 
to improve vision is unlikely to aid in your quest for better vision. In fact, eating too many carrots can cause 
your skin to appear yellow or yellow-orange due to a build-up of blood carotene levels.

[Do Carrots Really Improve Eyesight? The Truth Behind the 
Myth](https://thefuturemed.com/do-carrots-really-improve-eyesight-the-truth-behind-the-myth/)
What carrots can't do: Improve eyesight beyond normal levels; Cure conditions like myopia (nearsightedness) or 
presbyopia; Restore damaged vision or eliminate the need for glasses; So while carrots are healthy, they won't give
you perfect vision unless your vision problems are due to a lack of vitamin A. Other Nutrients Essential for Eye 
Health

[20 Benefits of Consuming Carrots for Vision Health - Scientific 
Origin](https://scientificorigin.com/20-benefits-of-consuming-carrots-for-vision-health)
This makes carrots a valuable food for preventing vision complications associated with diabetes. 16. Aids in 
Healing Eye Injuries. Carrots contain a variety of nutrients that support healing and recovery from eye injuries or
surgeries. Vitamin A promotes tissue repair, while the antioxidants in carrots help reduce inflammation and protect
the ...

[Are carrots really good for your eyes? - All About 
Vision](https://www.allaboutvision.com/eye-care/vision-health/nutrition/carrots-and-vision/)
Do carrots help your eyesight? Carrots contain a wide array of vitamins, including vitamin A, which has been known 
to improve eye health.In other words, there is truth to the claim that carrots are good for your eyes.. Vitamin A 
is known to benefit your eye health by:. Protecting your corneas; Decreasing the risk of cataracts and age-related 
macular degeneration

[Does carrot improve your vision? | SiOWfa16: Science in Our World 
...](https://sites.psu.edu/siowfa16/2016/11/26/does-carrot-improve-your-vision/)
Bottom Line: Many studies have showed strong evidence that carrots do not improve eyesight or vision if your 
eyesight is normal; however, studies showed that carrots do help maintain your vision indirectly, due to the 
beta-carotene component, which helps your body produce more vitamin A to help you see at night.

[Do carrots improve eyesight? The truth behind the 
myth](https://www.ophthalmologistsinus.com/does-eating-carrots-improve-eyesight-we-debunk-the-myth/)
Eating carrots is healthy, but it won't give you perfect eyesight. In conclusion, although carrots are a nutritious
food and have benefits for eye health, they are not a magic solution to improve vision. Maintaining a varied and 
balanced diet, along with good visual habits, is the best way to take care of your eyesight in the long run.

[Does Eating Carrots Improve Your Vision? - 
Snopes.com](https://www.snopes.com/fact-check/does-eating-carrots-improve-vision/)
While carrots are a good source of vitamin A (which is important

[Step 1: Duration 3.28 seconds]

'Carrots provide vitamin A essential for eye health, but do not significantly improve vision beyond preventing deficiencies.'

You can see that the answer of this example is judged as "not toxic".

![LLM-as-a-Judge Evaluation Score](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/llm-as-a-judge-score.png)

#### 6. Observability Metrics Overview

All of these metrics can be visualized together in dashboards. This enables you to quickly see how your agent performs across many sessions and helps you to track quality metrics over time.

![Observability metrics overview](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/langfuse-dashboard.png)

## Offline Evaluation

Online evaluation is essential for live feedback, but you also need **offline evaluation**—systematic checks before or during development. This helps maintain quality and reliability before rolling changes into production.

### Dataset Evaluation

In offline evaluation, you typically:
1. Have a benchmark dataset (with prompt and expected output pairs)
2. Run your agent on that dataset
3. Compare outputs to the expected results or use an additional scoring mechanism

Below, we demonstrate this approach with the [GSM8K dataset](https://huggingface.co/datasets/gsm8k), which contains math questions and solutions.

In [17]:
import pandas as pd
from datasets import load_dataset

# Fetch GSM8K from Hugging Face
dataset = load_dataset("openai/gsm8k", 'main', split='train')
df = pd.DataFrame(dataset)
print("First few rows of GSM8K dataset:")
print(df.head())

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

First few rows of GSM8K dataset:
                                            question  \
0  Natalia sold clips to 48 of her friends in Apr...   
1  Weng earns $12 an hour for babysitting. Yester...   
2  Betty is saving money for a new wallet which c...   
3  Julie is reading a 120-page book. Yesterday, s...   
4  James writes a 3-page letter to 2 different fr...   

                                              answer  
0  Natalia sold 48/2 = <<48/2=24>>24 clips in May...  
1  Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...  
2  In the beginning, Betty has only 100 / 2 = $<<...  
3  Maila read 12 x 2 = <<12*2=24>>24 pages today....  
4  He writes each friend 3*2=<<3*2=6>>6 pages a w...  


Next, we create a dataset entity in Langfuse to track the runs. Then, we add each item from the dataset to the system. (If you’re not using Langfuse, you might simply store these in your own database or local file for analysis.)

In [18]:
from langfuse import get_client
langfuse = get_client()

langfuse_dataset_name = "gsm8k_dataset_huggingface"

# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="GSM8K benchmark dataset uploaded from Huggingface",
    metadata={
        "date": "2025-03-10",
        "type": "benchmark"
    }
)

Dataset(id='cmciyuqd708lvad07z56j89r9', name='gsm8k_dataset_huggingface', description='GSM8K benchmark dataset uploaded from Huggingface', metadata={'date': '2025-03-10', 'type': 'benchmark'}, project_id='cmc0dg89z03nyad07plstps1n', created_at=datetime.datetime(2025, 6, 30, 10, 38, 50, 108000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 6, 30, 10, 38, 50, 108000, tzinfo=datetime.timezone.utc))

In [19]:
for idx, row in df.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["answer"]},
        metadata={"source_index": idx}
    )
    if idx >= 9: # Upload only the first 10 items for demonstration
        break

![Dataset items in Langfuse](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/example-dataset.png)

#### Running the Agent on the Dataset

We define a helper function `run_smolagent()` that:
1. Starts a Langfuse span
2. Runs our agent on the prompt
3. Records the trace ID in Langfuse

Then, we loop over each dataset item, run the agent, and link the trace to the dataset item. We can also attach a quick evaluation score if desired.

In [20]:
from opentelemetry.trace import format_trace_id
from smolagents import (CodeAgent, InferenceClientModel, LiteLLMModel)
from langfuse import get_client

langfuse = get_client()


# Example: using InferenceClientModel or LiteLLMModel to access openai, anthropic, gemini, etc. models:
model = GroqSmolModel()

agent = CodeAgent(
    tools=[],
    model=model,
    add_base_tools=True
)

dataset_name = "gsm8k_dataset_huggingface"
current_run_name = "smolagent-notebook-run-01" # Identifies this specific evaluation run

# Assume 'run_smolagent' is your instrumented application function
def run_smolagent(question):
    with langfuse.start_as_current_generation(name="qna-llm-call") as generation:
        # Simulate LLM call
        result = agent.run(question)

        # Update the trace with the input and output
        generation.update_trace(
            input= question,
            output=result,
        )

        return result

dataset = langfuse.get_dataset(name=dataset_name) # Fetch your pre-populated dataset

for item in dataset.items:

    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata={"model_provider": "Hugging Face", "temperature_setting": 0.7},
        run_description="Evaluation run for GSM8K dataset"
    ) as root_span: # root_span is the root span of the new trace for this item and run.
        # All subsequent langfuse operations within this block are part of this trace.

        # Call your application logic
        generated_answer = run_smolagent(question=item.input["text"])

        print(item.input)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tina makes $18.00 an hour.  If she works more than 8 hours per shift, she is eligible for overtime, which is    │
│ paid by your hourly wage + 1/2 your hourly wage.  If she works 10 hours every day for 5 days, how much money    │
│ does she make?                                                                                                  │
│                                                                                                                 │
╰─ GroqSmolModel -  ──────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

=== Final messages_payload ===
{'role': <MessageRole.SYSTEM: 'system'>, 'content': '[{\'type\': \'text\', \'text\': \'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \\\'Thought:\\\', \\\'<code>\\\', and \\\'Observation:\\\' sequences.\\n\\nAt each step, in the \\\'Thought:\\\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\\nThen in the \\\'<code>\\\' sequence, you should write the code in simple Python. The code sequence must end with \\\'</code>\\\' sequence.\\nDuring each intermediate step, you can use \\\'print()\\\' to save whatever important information you will then need.\\nThese print outputs will then appear in the \\\'Obser

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  regular_hourly_wage = 18                                                                                         
  hours_per_day = 10                                                                                               
  days = 5                                                                                                         
                                                                                                                   
  regular_hours_per_day = 8                                                                                        
  overtime_hours_per_day = hours_per_day - regular_hours_per_day                                                   
                                                                                                                   
  daily_regular_pay = regular_hours_per_day * regular_hourly_wage                                                  
  overtime_rate = regular_hourly_wage * 1.5                                                                        
  daily_overtime_pay = overtime_hours_per_day * overtime_rate                                                      
                                                                                                                   
  total_daily = daily_regular_pay + daily_overtime_pay                                                             
  total_earnings = total_daily * days                                                                              
                                                                                                                   
  final_answer(total_earnings)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 990.0

...

=== Final messages_payload ===
{'role': <MessageRole.SYSTEM: 'system'>, 'content': '[{\'type\': \'text\', \'text\': \'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \\\'Thought:\\\', \\\'<code>\\\', and \\\'Observation:\\\' sequences.\\n\\nAt each step, in the \\\'Thought:\\\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\\nThen in the \\\'<code>\\\' sequence, you should write the code in simple Python. The code sequence must end with \\\'</code>\\\' sequence.\\nDuring each intermediate step, you can use \\\'print()\\\' to save whatever important information you will then need.\\nThese print outputs will then appear in the \\\'Obser

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  april = 48                                                                                                       
  may = april / 2                                                                                                  
  total = april + may                                                                                              
  final_answer(int(total))                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 72

[Step 1: Duration 28.28 seconds]

{'text': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'}


You can repeat this process with different:
- Models (OpenAI GPT, local LLM, etc.)
- Tools (search vs. no search)
- Prompts (different system messages)

Then compare them side-by-side in your observability tool:

![Dataset run overview](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/dataset_runs.png)
![Dataset run comparison](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/dataset-run-comparison.png)


## Final Thoughts

In this notebook, we covered how to:
1. **Set up Observability** using smolagents + OpenTelemetry exporters
2. **Check Instrumentation** by running a simple agent
3. **Capture Detailed Metrics** (cost, latency, etc.) through an observability tools
4. **Collect User Feedback** via a Gradio interface
5. **Use LLM-as-a-Judge** to automatically evaluate outputs
6. **Perform Offline Evaluation** with a benchmark dataset

🤗 Happy coding!